# Introduksjon

## Forfatter(e)
1. Magne Furuholmen Myhren

## Bestilling

Hei igjen,
Er tilsvarende tall for som for 2020, for 2021-årgangen vi behøver for farlig avfall. Helst så fort som mulig 😊
Mvh Terje Landsem


# Setup

## Velger årgang og delregister
Legger også til filsti til rådata fra NPR og avtalsespesialister

In [ ]:
aargang = 2022
siste_to_siffer_aargang = str(aargang)[-2:]

aar_for = aargang - 1                          # året før
siste_to_siffer_aar_for = str(aar_for)[-2:]

lagre_filer = False                            # sett til True, hvis du vil lagre droplistene
sti_til_lagring = ""                           # eventell sti til lagringsmappe. 
                                               # Lagres i denne mappen hvis tom

## Importerer pakker og setter innstillinger

In [ ]:
import numpy as np
import pandas as pd
import cx_Oracle
import getpass
import os
import matplotlib.pyplot as plt
import requests
from pandas.io.json import json_normalize #package for flattening json in pandas df

# Fjerner begrensning på antall rader og kolonner som vises av gangen
pd.set_option("display.max_columns", None)
pd.set_option('display.max_rows', 300)
pd.set_option('display.max_colwidth', None)

# Unngå standardform i output
pd.set_option('display.float_format', lambda x: '%.5f' % x)

## Tilgang oracle

In [ ]:
conn = cx_Oracle.connect(getpass.getuser()+"/"+getpass.getpass(prompt='Oracle-passord: ')+"@DB1P")

# Henter data

In [ ]:
sti = "/ssb/stamme01/helsep/wk24/g2020_sysskostra.parquet"
df = pd.read_parquet(sti)

In [ ]:
cols = "VIRK_ID_SSB, VIRK_NAVN, FRTK_ID_SSB, FRTK_NAVN, VIRK_NACE1_SN07, ARB_AARSVERK, PERS_LANGEFRA_ANDEL, VIRK_HO_MERKE".split(", ")
df1 = df[cols].copy()

In [ ]:
df1['ARSVERK_EKSKL'] = df['ARB_AARSVERK']-(df['ARB_AARSVERK']*df['PERS_LANGEFRA_ANDEL'])

In [ ]:
naring = ["86.102", "86.906", "86.101"]
df1 = df1.loc[df1['VIRK_NACE1_SN07'].isin(naring)]

In [ ]:
df1 = df1.loc[df1['VIRK_HO_MERKE'].isin(["SPES"])]

In [ ]:
sumert = df1.groupby('VIRK_NACE1_SN07').sum()

In [ ]:
s1 = "86.101,86.102,86.906".split(",")
s2 = "351.8542,54.4438,1132.1657".split(",")
s3 = "69019.97544,2371.582708,8281.057277".split(",")

In [ ]:
res = pd.DataFrame([s1,s2,s3]).T
res.columns=["VIRK_NACE1_SN07","virk_utenfor_delreg", "virk_i_delreg"]
res.virk_utenfor_delreg = pd.to_numeric(res.virk_utenfor_delreg, errors='coerce')
res.virk_i_delreg = pd.to_numeric(res.virk_i_delreg, errors='coerce')

In [ ]:
sumert = pd.merge(sumert,res,on="VIRK_NACE1_SN07")

In [ ]:
sumert['diff'] = sumert.ARSVERK_EKSKL - sumert.virk_i_delreg
sumert

In [ ]:
df1 = df1.drop(columns=['ARB_AARSVERK','PERS_LANGEFRA_ANDEL'])

In [ ]:
virk_i_delreg_df = df1.groupby(['VIRK_ID_SSB','VIRK_NAVN','FRTK_ID_SSB','FRTK_NAVN','VIRK_NACE1_SN07']).sum()

In [ ]:
virk_i_delreg_df = virk_i_delreg_df.reset_index()


In [ ]:
virk_i_delreg_df.columns = ["ORGNR", "VIRK_NAVN", "ORGNR_FORETAK_SYSK", "FRTK_NAVN", "naceny", "sum_aarsverk_ekskl"]

In [ ]:
virk_i_delreg_df['sum_aarsverk_ekskl'] = virk_i_delreg_df['sum_aarsverk_ekskl'].astype(str)

In [ ]:
virk_i_delreg_df['sum_aarsverk_ekskl'] = virk_i_delreg_df['sum_aarsverk_ekskl'].apply(lambda x: x.replace(".", ","))

# Eksportering til .csv-filer og rapport

In [ ]:
# def rapport(skj):
#     rader = eval(skj).shape[0]
#     kolonner = eval(skj).shape[1]
#     rader_med_missing = rader - eval(skj).dropna().shape[0]
#     skl = 100 * "-"
#     s = f"{skl}\n{skj}\t Rader: {rader} \t Kolonner: {kolonner} \t Rader som inneholder minst en missing value: {rader_med_missing} "
#     return s

# def lagre_dropliste_csv(skj):
#     dato_idag = pd.Timestamp("today").strftime("%d%m%y")
#     filnavn = "Dropliste_" + skj + "_" + str(aargang) + "_" + dato_idag + ".csv"
#     eval(skj).to_csv(sti_til_lagring + filnavn, sep=";", encoding='latin1', index=False)
#     print(filnavn, " lagret")

In [ ]:
# for x in skjemaer_til_droplister:
#     print(rapport(x))

In [ ]:
# if lagre_filer:
#     for x in skjemaer_til_droplister:
#         lagre_dropliste_csv(x)